In [84]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [85]:
sample_submissiom = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [86]:
print(train_data.describe())

In [87]:
col_na = []
for i in train_data.columns.values:
    if len(train_data[train_data[i].isna()])>38:
        col_na.append(i)
        print(f'{i}',len(train_data[train_data[i].isna()]))
col_na.append('Id')
col_name = list(set(train_data.columns.values) - set(col_na))
train_data1 = train_data[col_name]
for i in col_name:
    train_data1 = train_data1[train_data1[i].notna()]
train_data1['SalePrice']

In [88]:
import matplotlib.pyplot as plt
import seaborn as sns
col_cor = train_data1.corr()['SalePrice'][abs(train_data1.corr()['SalePrice'])>=0.6].index
sns.heatmap(train_data1[col_cor].corr(), annot = True)
plt.show()
print(col_cor)

In [89]:
col_res = list(set(col_cor)-set(['1stFlrSF','TotRmsAbvGrd','GarageArea','SalePrice']))
print(col_res)

In [90]:
for i in col_res:
    plt.scatter(train_data1[i],train_data1['SalePrice'])
    plt.title(i)
    plt.show()

1stFlrSf and TotalBsmtsF -> TotalBsmtsF
TotRms and GirLiv -> GirLiv
GCars and Aarea -> GCars

In [91]:
train_data1['SalePrice'].describe()

In [92]:
color = []
for i in train_data1['SalePrice']:
    if i <= 131475:
        color.append('red')
    elif i <= 215000:
        color.append('yellow')
    else:
        color.append('green')

In [93]:
import matplotlib.pyplot as plt
plt.scatter(train_data1['GarageCars'],train_data1['OverallQual'],c=color,alpha = 0.5)

plt.show()

In [94]:
for i in col_res:
    print(test_data[test_data[i].isna()][col_res])

In [95]:
test_data['GarageCars'] = test_data['GarageCars'].fillna(test_data['GarageCars'].mean())
test_data['TotalBsmtSF'] = test_data['TotalBsmtSF'].fillna(test_data['TotalBsmtSF'].mean())

In [99]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_data1[col_res], train_data1['SalePrice'])
melb_preds = forest_model.predict(test_data[col_res])
print(mean_absolute_error(sample_submissiom['SalePrice'], melb_preds))
print(melb_preds[0:5],sample_submissiom['SalePrice'][0:5])

In [97]:

sns.lineplot(data = sample_submissiom['SalePrice'])
sns.lineplot(data = melb_preds)
plt.show()

In [101]:
output = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': melb_preds})
output.to_csv('my_submission.csv', index=False)
print(output)